# Домашнее задание

Работа с vllm

**Цель:**

В данном домашнем задании вы будете запускать языковые модели с помощью фреймворка vllm и работать с языковыми моделями, запущенными через vllm.

## 1. Загрузка датасета и формулировка промпта

Что сделать:

- Выберите небольшой (≤ 10 000 примеров) датасет с HuggingFace для генеративной или классификационной задачи. Пример: HuggingFaceH4/MATH-500.
- Загрузите весь датасет без разбивки на тренировочную и тестовую части.
- Придумайте и пропишите prompt, который будет понятен модели и направит её отвечать в нужном формате (число, выражение в LaTeX и т. д.).

In [1]:
!pip install evaluate
!pip install bert_score
!pip install vllm

In [2]:
from tqdm.auto import tqdm, trange
import pandas as pd
import torch

from datasets import load_dataset

from evaluate import load

In [3]:
data_files = {'test': './test.jsonl'}
data = load_dataset('json', data_files=data_files)
data

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['problem', 'solution', 'answer', 'subject', 'level', 'unique_id'],
        num_rows: 500
    })
})

In [4]:
data['test'][0]['problem']

'Convert the point $(0,3)$ in rectangular coordinates to polar coordinates.  Enter your answer in the form $(r,\\theta),$ where $r > 0$ and $0 \\le \\theta < 2 \\pi.$'

In [5]:
data['test'][0]['answer']

'\\left( 3, \\frac{\\pi}{2} \\right)'

$\left( 3, \frac{\pi}{2} \right)$

## 2. Установка vllm и подъем модели

Что сделать:

- Установите библиотеку:

pip install vllm

- Выберите инструктивную модель с chat_template (например, Qwen/Qwen2.5-0.5B-Instruct).
- Инициализируйте её в ноутбуке:

from vllm import LLM
llm = LLM(model="Qwen/Qwen2.5-0.5B-Instruct")

In [6]:
from vllm import LLM, SamplingParams

INFO 10-02 13:45:51 [__init__.py:216] Automatically detected platform cuda.


In [7]:
#LLM?

In [8]:
base_model = "Qwen/Qwen2.5-0.5B-Instruct"

In [9]:
llm = LLM(
    model=base_model,
    dtype=torch.float16,
    tensor_parallel_size=1,
    gpu_memory_utilization=0.9,
    seed=13,
    tokenizer=base_model,
)

INFO 10-02 13:45:54 [utils.py:328] non-default args: {'tokenizer': 'Qwen/Qwen2.5-0.5B-Instruct', 'dtype': torch.float16, 'seed': 13, 'disable_log_stats': True, 'model': 'Qwen/Qwen2.5-0.5B-Instruct'}


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

INFO 10-02 13:46:17 [__init__.py:742] Resolved architecture: Qwen2ForCausalLM


`torch_dtype` is deprecated! Use `dtype` instead!


...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

WARNING 10-02 13:46:21 [__init__.py:2974] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reasons: CUDA is initialized
INFO 10-02 13:47:35 [llm.py:295] Supported_tasks: ['generate']
INFO 10-02 13:47:35 [__init__.py:36] No IOProcessor plugins requested by the model


In [10]:
llm

## 3. Генерация через generate

Что сделать:

- Используйте модуль SamplingParams для передачи параметров генерации (выберите параметры, которые вам кажутся оптимальными)
- Передайте списком все вопросы датасета в метод llm.generate без шаблона (не забудьте про параметры генерации):

outputs = llm.generate(requests, use_tqdm=True)

- Извлеките чистые тексты ответов.

In [11]:
sampling_params = SamplingParams(max_tokens=512, temperature=0.0)
sampling_params

SamplingParams(n=1, presence_penalty=0.0, frequency_penalty=0.0, repetition_penalty=1.0, temperature=0.0, top_p=1.0, top_k=0, min_p=0.0, seed=None, stop=[], stop_token_ids=[], bad_words=[], include_stop_str_in_output=False, ignore_eos=False, max_tokens=512, min_tokens=0, logprobs=None, prompt_logprobs=None, skip_special_tokens=True, spaces_between_special_tokens=True, truncate_prompt_tokens=None, guided_decoding=None, extra_args=None)

In [12]:
questions_list = [item for item in data['test']['problem']]

In [13]:
questions_list[1]

'Define\n\\[p = \\sum_{k = 1}^\\infty \\frac{1}{k^2} \\quad \\text{and} \\quad q = \\sum_{k = 1}^\\infty \\frac{1}{k^3}.\\]Find a way to write\n\\[\\sum_{j = 1}^\\infty \\sum_{k = 1}^\\infty \\frac{1}{(j + k)^3}\\]in terms of $p$ and $q.$'

In [14]:
out = llm.generate(questions_list[0], sampling_params=sampling_params, use_tqdm=True)

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [15]:
out[0].outputs[0].text

'\n\nTo convert the point \\((0,3)\\) from rectangular coordinates to polar coordinates, we need to find the radius \\(r\\) and the angle \\(\\theta\\).\n\nThe radius \\(r\\) is the distance from the origin to the point \\((0,3)\\). This distance is given by the formula:\n\\[\nr = \\sqrt{x^2 + y^2}\n\\]\nSubstituting \\(x = 0\\) and \\(y = 3\\), we get:\n\\[\nr = \\sqrt{0^2 + 3^2} = \\sqrt{9} = 3\n\\]\n\nNext, we need to find the angle \\(\\theta\\). The angle \\(\\theta\\) is the angle that the line from the origin to the point \\((0,3)\\) makes with the positive \\(x\\)-axis. This angle can be found using the arctangent function, but we also need to consider the quadrant in which the point \\((0,3)\\) lies. Since the point \\((0,3)\\) is in the fourth quadrant, the angle \\(\\theta\\) is:\n\\[\n\\theta = \\tan^{-1}\\left(\\frac{y}{x}\\right) = \\tan^{-1}\\left(\\frac{3}{0}\\right)\n\\]\nHowever, \\(\\frac{3}{0}\\) is undefined, which means the point \\((0,3)\\) is actually at the ori

Попытка запуска процесса инференса в пакетном режиме приводит к падению vllm, предполагаю, что не хватает памяти... Поэтому ответы от модели будем получать по одному в цикле, а также ограничимся 20 сэмплами из набора данных.

In [16]:
SAMPLES_NUM=20

In [17]:
out_list = []
for question in tqdm(questions_list[:SAMPLES_NUM]):
  out = llm.generate(question, sampling_params=sampling_params)
  out_list.append(out[0].outputs[0].text)

  0%|          | 0/20 [00:00<?, ?it/s]

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

...

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [18]:
#out = llm.generate(questions_list, sampling_params=sampling_params, use_tqdm=True)

In [19]:
out_list[:3]

['\n\nTo convert the point \\((0,3)\\) from rectangular coordinates to polar coordinates, we need to find the radius \\(r\\) and the angle \\(\\theta\\).\n\nThe radius \\(r\\) is the distance from the origin to the point \\((0,3)\\). This distance is given by the formula:\n\\[\nr = \\sqrt{x^2 + y^2}\n\\]\nSubstituting \\(x = 0\\) and \\(y = 3\\), we get:\n\\[\nr = \\sqrt{0^2 + 3^2} = \\sqrt{9} = 3\n\\]\n\nNext, we need to find the angle \\(\\theta\\). The angle \\(\\theta\\) is the angle that the line from the origin to the point \\((0,3)\\) makes with the positive \\(x\\)-axis. This angle can be found using the arctangent function, but we also need to consider the quadrant in which the point \\((0,3)\\) lies. Since the point \\((0,3)\\) is in the fourth quadrant, the angle \\(\\theta\\) is:\n\\[\n\\theta = \\tan^{-1}\\left(\\frac{y}{x}\\right) = \\tan^{-1}\\left(\\frac{3}{0}\\right)\n\\]\nHowever, \\(\\frac{3}{0}\\) is undefined, which means the point \\((0,3)\\) is actually at the or

## 4. Оценка метрик для generate

Выбор метрик:

- **BLEU** (Bilingual Evaluation Understudy) больше подходит, когда важно оценить точность (precision) перевода, как например, в задаче машинного перевода, для которой данная метрика и была создана изначально. Для оценки точности решения математической задачи подойдет дучше, нежели ориентированная на оценку семантической близости метрика ROUGE;

- **Exact_match** оценивает точное совпадение для каждой пары (1 - если совпадение точное и 0 - в остальных случаях) и усредняет по количеству пар. Учитывая, что у математической задачи должен быть один правильный ответ, то также подойдет к выбранному набору данных.

In [20]:
results = {}

In [21]:
bleu = load("bleu")

In [22]:
exact_match = load("exact_match")

In [23]:
bleu_metric = bleu.compute(predictions=out_list, references=data['test']['solution'][:SAMPLES_NUM])
bleu_metric

{'bleu': 0.12429971527181119,
 'precisions': [0.2842834178310869,
  0.1478864509731003,
  0.09133838697651299,
  0.06216519247539554],
 'brevity_penalty': 1.0,
 'length_ratio': 2.192193006234752,
 'translation_length': 8087,
 'reference_length': 3689}

In [24]:
em_metric = exact_match.compute(predictions=out_list, references=data['test']['solution'][:SAMPLES_NUM])
em_metric

{'exact_match': np.float64(0.0)}

In [25]:
def quality(predictions, references):
  bleu_metric = bleu.compute(predictions=predictions, references=references)
  print(bleu_metric)

  em_metric = exact_match.compute(predictions=predictions, references=references)
  print(em_metric)

  return [bleu_metric['bleu'], em_metric['exact_match']]

In [26]:
results['raw questions'] = quality(predictions=out_list, references=data['test']['solution'][:SAMPLES_NUM])

{'bleu': 0.12429971527181119, 'precisions': [0.2842834178310869, 0.1478864509731003, 0.09133838697651299, 0.06216519247539554], 'brevity_penalty': 1.0, 'length_ratio': 2.192193006234752, 'translation_length': 8087, 'reference_length': 3689}
{'exact_match': np.float64(0.0)}


In [27]:
pd.DataFrame(results, index=['bleu', 'exact_match']).T

,bleu,exact_match
raw questions,0.1243,0.0


## 5. Генерация через chat с chat_template

Что сделать:

- Оформите каждый запрос в формате списка словарей, подходящем под chat_template:

messages = [
{"role": "system", "content": "Тут может быть систем-промпт, если вы решите его добавить"},
{"role": "user", "content": question}
]

- Вызовите llm.chat и получите ответы.
- Извлеките текст из каждого сообщения.`

In [28]:
def make_short_template(text):
    return [[
        {
            "role": "system",
            "content": "You are helpful math expert tasked with resolving math problems. Provide only the final answer, without explanation or any additinal details. Use LaTeX format, if appropriate'"
        },
        {
            "role": "user",
            "content": text + ' Provide only the final answer, without explanation or any additinal details. Use LaTeX format, if appropriate'
        }
    ]]

In [29]:
def make_template(text):
    return [[
        {
            "role": "system",
            "content": "You are helpful math expert tasked with resolving math problems."
        },
        {
            "role": "user",
            "content": text + ' Provide brief explanation and final answer, use LaTeX format, if appropriate'
        }
    ]]

In [30]:
test = data['test'][0]['problem'] #'Сколько будет 2+2 ?'
template = make_short_template(test)
template[0], len(template)

([{'role': 'system',
   'content': "You are helpful math expert tasked with resolving math problems. Provide only the final answer, without explanation or any additinal details. Use LaTeX format, if appropriate'"},
  {'role': 'user',
   'content': 'Convert the point $(0,3)$ in rectangular coordinates to polar coordinates.  Enter your answer in the form $(r,\\theta),$ where $r > 0$ and $0 \\le \\theta < 2 \\pi.$ Provide only the final answer, without explanation or any additinal details. Use LaTeX format, if appropriate'}],
 1)

In [31]:
sampling_params = SamplingParams(max_tokens=512, temperature=0.0)

In [32]:
out = llm.chat(template, sampling_params=sampling_params, use_tqdm=True)

INFO 10-02 13:50:34 [chat_utils.py:538] Detected the chat template content format to be 'string'. You can set `--chat-template-content-format` to override this.


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [33]:
out[0].outputs[0].text

'$1$, $\\frac{\\pi}{2}$'

In [41]:
data['test'][0]['answer']

'\\left( 3, \\frac{\\pi}{2} \\right)'

Модель последовала short_template и дала короткий, но **неверный ответ:** $1, \frac{\pi}{2}$. Первая координата должна быть равна 3. Это, кстати, распространенный эффект, когда модель с рассуждениями (reasoning) дает более точные ответы в математических задачах, чем без него. Для корректного сравнения с предыдущими результатами, мы не будем заставлять модель давать только итоговый ответ.

In [35]:
out_chat_list = []
for question in tqdm(questions_list[:SAMPLES_NUM]):
  out = llm.chat(make_template(question), sampling_params=sampling_params)
  out_chat_list.append(out[0].outputs[0].text)

  0%|          | 0/20 [00:00<?, ?it/s]

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

...

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

## 6. Оценка метрик для chat

Что сделать:

- Примените те же метрики, что и в пункте 4, к ответам из пункта 5.
- Сохраните и сравните результаты.

In [38]:
results['chat template'] = quality(predictions=out_chat_list, references=data['test']['solution'][:SAMPLES_NUM])

{'bleu': 0.11941888722875901, 'precisions': [0.272596499640374, 0.14215332852679644, 0.08781016622500602, 0.05976817193914513], 'brevity_penalty': 1.0, 'length_ratio': 2.261317430197886, 'translation_length': 8342, 'reference_length': 3689}
{'exact_match': np.float64(0.0)}


In [39]:
pd.DataFrame(results, index=['bleu', 'exact_match']).T

,bleu,exact_match
raw questions,0.124300,0.0
chat template,0.119419,0.0


In [42]:
out_chat_list[0]

'To convert the point \\((0,3)\\) from rectangular coordinates to polar coordinates, we need to find the radius \\(r\\) and the angle \\(\\theta\\).\n\n### Step 1: Calculate the radius \\(r\\)\nThe radius \\(r\\) in polar coordinates is given by the formula:\n\\[\nr = \\sqrt{x^2 + y^2}\n\\]\nwhere \\(x\\) and \\(y\\) are the coordinates of the point. For the point \\((0,3)\\):\n\\[\nr = \\sqrt{0^2 + 3^2} = \\sqrt{9} = 3\n\\]\n\n### Step 2: Calculate the angle \\(\\theta\\)\nThe angle \\(\\theta\\) in polar coordinates is given by the formula:\n\\[\n\\theta = \\tan^{-1}\\left(\\frac{y}{x}\\right)\n\\]\nHowever, we need to consider the quadrant in which the point \\((0,3)\\) lies. The point \\((0,3)\\) is in the first quadrant, where both \\(x\\) and \\(y\\) are positive. Therefore, we use the arctangent function:\n\\[\n\\theta = \\tan^{-1}\\left(\\frac{3}{0}\\right)\n\\]\nThis results in an undefined value because division by zero is not allowed. However, we can still find the angle in 

## 7. Сравнительный анализ и выводы

Что сделать:

- Сравните все полученные метрики: generate vs. chat.
- Сделайте вывод, какой режим (без шаблона или с chat_template) оказался более эффективным для вашей модели и задачи.
- Проанализируйте, насколько ваш первоначальный промпт был удачен и что можно улучшить.

Итоговая таблица:

In [40]:
pd.DataFrame(results, index=['bleu', 'exact_match']).T.sort_values(by=['bleu'], ascending=False)

,bleu,exact_match
raw questions,0.124300,0.0
chat template,0.119419,0.0


- Результаты совпадают, с точностью до округления до второго знака. В предыдущий запуск результаты с chat_template были чуть выше.

- Как мы видели, без рассуждений (short_template), модель дала неверный ответ на первый вопрос. Это распространенный эффект, когда модель с рассуждениями (reasoning) дает более точные ответы в математических задачах.

- С рассуждениями результаты в режиме chat, совпадают с результатами режима generate, что в общем-то логично. Для получения более высоких результатов имеет смысл взять модель с большим числом параметров, специально обученную на математических задачах, например Qwen2.5-Math-7B-Instruct, а также провести тесты на полном наборе данных.